In [1]:
import pandas as pd
import numpy as np
import nltk
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn import svm
from sklearn.model_selection import GridSearchCV, train_test_split
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

In [2]:
df = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv",encoding='ISO-8859-1')
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [3]:
encoder = LabelEncoder()

df['label']  = encoder.fit_transform(df['v1'])



df = df.drop_duplicates(keep= 'first')


In [4]:
df = df.drop(['v1'], axis = 1)

In [5]:
df['SMS'] = df['v2']

df = df.drop(['v2','Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis = 1)

In [6]:
df

,label,SMS
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [7]:
def get_importantFeatures(sent):
    sent = sent.lower()
    returnList = []
    sent = nltk.word_tokenize(sent)
    for i in sent:
        if i.isalnum():
            returnList.append(i)
    return returnList

def removing_stopWords(sent):
    returnList = []
    for i in sent:
        if i not in nltk.corpus.stopwords.words('english') and i not in string.punctuation:
            returnList.append(i)
    return returnList

def porter_stem(sent):
    returnList = []
    for i in sent:
        returnList.append(ps.stem(i))
    return " ".join(returnList)

In [8]:
df['imp_feature'] = df['SMS'].apply(get_importantFeatures)

df['imp_feature'] = df['imp_feature'].apply(removing_stopWords)

df['imp_feature'] = df['imp_feature'].apply(porter_stem)

In [9]:
x = df['imp_feature']
y = df['label']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.25, random_state= 42)

In [10]:
tfidf = TfidfVectorizer()

feature = tfidf.fit_transform(x_train)

tuned_parameters = {'kernel': ['linear', 'rbf'], 'gamma': [1e-3,1e-4], 'C': [1, 10, 100, 1000]}

model = GridSearchCV(svm.SVC(), tuned_parameters)
model.fit(feature, y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                         'kernel': ['linear', 'rbf']})

In [11]:
y_pred = tfidf.transform(x_test)
print("Accuracy: ", model.score(y_pred, y_test))

Accuracy:  0.9806651198762568
